# DiCE ML Comparison with Optimal Point Method 

We compare the DiCE Model-agnostic methods with the optimal point method in this notebook. 

1. First, we import DiCE ML model-agnostic methods 
2. Second, we import the packaged files needed to run the "Optimal Point" Methodology 
3. Third, we run the experiments and compare the results at the end using different models such as SVM and random forest classifier.
4. Finally, we compare the runtimes of DiCE and the Optimal Point methodology

Note: Running experiments for the adult income can take hours. Please be mindful of the runtime for these experiments.

# Step 1: Importing DiCE ML and helper functions 

Below we import DiCE ML and their relevant helper functions. We import the sci-kit learn library and some of their necessary methods to make sure that we can run the experiments. 

In [ ]:
# import DiCE
import dice_ml

from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn import svm

import pandas as pd
import numpy as np
import json 
import datetime  

import warnings
import random
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import urllib.request
from urllib.request import urlopen
import ssl
import json
ssl._create_default_https_context = ssl._create_unverified_context
from dice_ml.utils import helpers


# Step 2: Import the necessary functionality to make Optimal Point methodology work 

We import many of the methods needed for the ```optimal_point()``` function to work as intended below. We import additional methods from ```binary_search_optimal_point().```

In [2]:
from files.common_functions import euclidean_distance, closest_border_point, closest_point, move_from_A_to_B_with_x1_displacement
from files.common_functions import get_multi_dim_border_points, det_constraints, real_world_constraints, constraint_bounds
from files.common_functions import balance_dataset, check_class_balance, convert_columns
from files.binary_search_optimal_point import multi_decision_boundary

In [3]:
# Loading cuML GPU acceleration library
%load_ext cuml.accel

In [4]:
df=pd.read_csv('../../toy_dataset.csv')
# SVM classifier with polynomial decision boundary
svm_classifier = svm.SVC(kernel='poly',C=10, degree=2, probability=True, random_state=1)

In [ ]:
def optimal_point(dataset, model, desired_class, original_class, chosen_row=-1, threshold=10000, point_epsilon=0.1, epsilon=0.01, constraints=[], delta=15, plot=False, step=0.5): 
    """
    Finds the closest point to the decision boundary from an undesired point,
    optionally constrained by real-world conditions.
    This essentially finds the counterfactual explanation for a given point by minimizing the distance to the given boundary.
    This method is important because it addresses a key problem with the original optimal_point() function where we generated an R^n dimensional grid that we would then have to iterate over. 
    The problem with iterating over such a grid is eventually that we will hit a memory error for high-dimensional features such as 20, 30 or 40 features. This will cause the function to crash. 
    Additionally, due to the exponential increase of the number of features to search, the grid will become infeasible to search (curse of dimensionality). 

    Parameters
    ----------
    dataset : pd.DataFrame
        Full dataset containing features and a final column with class labels.
    
    model : sklearn-like classifier
        A binary classification model with a `.fit()` and `.predict()` method.
    
    desired_class : int or label
        The target class we want the corrected point to belong to.
    
    original_class : int or label
        The actual class label of the undesired point.
    
    chosen_row :  int 
        The selected row of the dataset to find the counterfactual explanation for
    
    threshold : int, optional
        Max number of decision boundary points to sample. Default is 10000.
    
    point_epsilon : float, optional
        Precision used to estimate decision boundary points. Default is 0.1.
    
    epsilon : float, optional
        Step size used when displacing a point toward the decision boundary. Default is 0.01.
    
    constraints : list, optional
        A list of real-world constraints on the features (e.g., ranges, logic constraints). Default is [].
    
    delta : int, optional
        Tolerances or maximum displacement for each continuous feature

    plot : boolean 
        Used as a parameter to determine whether to plot the results or not

    Returns
    -------
    np.ndarray
        A corrected point that satisfies the class change and real-world constraints.

    Raises
    ------
    Exception
        If the number of constraints exceeds the number of features.

    Notes
    -----
    - This function trains the model on the provided dataset, generates boundary points using
      `find_decision_boundary`, applies constraints, and finds the closest optimal point.
    - Assumes binary classification and relies on external functions like `real_world_constraints`,
      `closest_point`, `move_from_A_to_B_with_x1_displacement`, etc., which must be defined elsewhere.
    - Includes plotting for visualization (e.g., boundary contours, points), which requires matplotlib.
    - The function blends boundary approximation with counterfactual generation, useful for explainable AI.
    - Print statements are for progress tracking; plotting is partially commented out but can be enabled.
    - Usage: Call with a dataset and model to generate counterfactuals, e.g., for model interpretation or optimization.

    Examples
    --------
    >>> import pandas as pd
    >>> from sklearn.linear_model import LogisticRegression
    >>> dataset = pd.DataFrame({'feat1': [0, 1, 2], 'feat2': [0, 1, 0], 'label': [0, 1, 0]})
    >>> model = LogisticRegression()
    >>> undesired_coords = [2, 0]  # Example point from class 0
    >>> optimal = optimal_point(dataset, model, desired_class=1, original_class=0, undesired_coords=undesired_coords)
    >>> print(optimal)  # e.g., array([[1.5, 0.5]])
    """

    # Convert categorical columns if needed (before balancing)
    inv_col_map = convert_columns(dataset)

    # Extract features and labels before balancing
    X_orig = dataset.iloc[:, :-1]
    
    # Save the original row's feature values
    undesired_coords = X_orig.iloc[chosen_row, :].copy()

    # Balance the dataset
    dataset = balance_dataset(df=dataset, target=dataset.columns[-1])
    
    if not check_class_balance(dataset, target=dataset.columns[-1]):
        raise RuntimeError("Failed to balance classes for binary classification")
    
    sampled_dataset = dataset.sample(n=min(dataset.shape[0], 20000))

    # Extract new training features/labels after balancing
    X_train = sampled_dataset.iloc[:, :-1]
    y_train = sampled_dataset.iloc[:, -1]
    # Train the model
    print("Fitting model...")
    model.fit(X_train, y_train)
    print("Model training complete.")

    # -------------------------------
    # STEP 2: Find decision boundary
    # -------------------------------
    print("boundary points started generation...")

    # This step uses binary interpolation to get points close to the decision boundary
    boundary_points = multi_decision_boundary(model, X_train, y_train,
                                             threshold=threshold, epsilon=point_epsilon)
    print("boundary points finished.")
    print(boundary_points.shape)
    # Detect categorical features (assumed as int columns)
    categorical_features = X_train.select_dtypes(include=['int32', 'int64', 'int8']).columns.tolist()

    # Round categoricals to int for discrete values
    for col in categorical_features: 
        boundary_points[col] = boundary_points[col].astype(int)

    # -------------------------------
    # STEP 3: Apply real-world constraints (optional)
    # -------------------------------
    # Reduce boundary points based on external rules (e.g., cost limits, physics constraints)
    contours_pd = real_world_constraints(points=boundary_points,
                                      undesired_coords=undesired_coords,
                                      constraints=constraints)
    undesired_datapt = np.reshape(undesired_coords, (1, -1))  # Reshape undesired point to 2D array

    # if plot:
    #     plt.plot(contours[:,0], contours[:,1], lw=0.5, color='red')  # Commented: Plot contours for visualization

    # -------------------------------
    # STEP 4: Find closest point on constrained boundary
     # -------------------------------
    if contours_pd is not None and desired_class != original_class: 
        contours = contours_pd.to_numpy()
        print("Finding the closest point from the contour line to the point...")
        contours_pd.reset_index(drop=True, inplace=True)
        optimal_datapt = closest_point(undesired_datapt, contour=contours)
        print("Found the closest point from the contour line to the point.")  # Note: Duplicate print, possibly a typo
        D = optimal_datapt - undesired_datapt  # Compute direction vector
        deltas = D * (1+epsilon)  # Scale by (1 + epsilon) to overshoot
        optimal_datapt = move_from_A_to_B_with_x1_displacement(undesired_datapt, optimal_datapt, deltas=deltas)
    elif desired_class == original_class or contours_pd is None: 
        # If we want to *stay within* the same class (more constrained)
        all_constrained_feats = [var for (var,_) in constraints]
        closest_boundedpt = None 
        vars = set(X_train.columns) - set(all_constrained_feats)
        cont_mutable_vars = [X_train.columns.get_loc(col) for col in vars]
        deltas, len_constr = det_constraints(datapt=undesired_datapt[0], vars=cont_mutable_vars, deltas=deltas)  # Determine constraints
        
        if len_constr > X_train.shape[1]:
            raise Exception("There cannot be more constraints than features")
        else:
            # All n dimensions are constrained, so generate an exact grid of boundary candidates
            bounded_contour_pts = get_multi_dim_border_points(center=undesired_datapt[0],
                                                              extents=deltas,
                                                              step=0.1)
            np_bounded_contour = np.array(bounded_contour_pts)  # Convert to NumPy array
            if plot:           
                x_values, y_values = np_bounded_contour[:, 0], np_bounded_contour[:, 1]  # Extract x/y for plotting
                plt.scatter(x_values, y_values, marker='o')  # Plot bounded points

            closest_boundedpt = closest_border_point(np_bounded_contour, contour=boundary_points)  # Find closest on border
            print(closest_boundedpt)
        D = closest_boundedpt - undesired_datapt  # Compute direction
        optimal_datapt = move_from_A_to_B_with_x1_displacement(undesired_datapt, closest_boundedpt, deltas=D)  # Move point
    # Plot original and optimal points with connecting line
    # if plot:
    #     plt.scatter(undesired_datapt[0][0], undesired_datapt[0][1], c = 'r')  # Plot undesired point
    #     plt.text(undesired_datapt[0][0]+0.002, undesired_datapt[0][1]+0.002, 'NH')  # Label 'NH' (e.g., Non-Healthy)
    #     plt.scatter(optimal_datapt[0][0], optimal_datapt[0][1], c = 'g')  # Plot optimal point (changed to green for distinction)
    #     plt.text(optimal_datapt[0][0]+0.002, optimal_datapt[0][1]+0.002, 'NH')  # Label 'H' (e.g., Healthy; adjusted from duplicate 'NH')
    #     plt.plot([undesired_datapt[0][0], optimal_datapt[0][0]], [undesired_datapt[0][1],optimal_datapt[0][1]], linestyle='--')  # Dashed line between points
    
    categorical_features = [col for col in inv_col_map.keys()]
    final_optimal_datapt = [] 

    for col in X_train.columns:
        if col in categorical_features: 
            idx = int(optimal_datapt[0,X_train.columns.get_loc(col)])
            final_optimal_datapt.append(inv_col_map[col][idx])
        else: 
            final_optimal_datapt.append(optimal_datapt[0,X_train.columns.get_loc(col)])
    query_instance = undesired_coords
    return dataset, model, query_instance, final_optimal_datapt, euclidean_distance(undesired_datapt, optimal_datapt), delta, boundary_points

In [6]:
def clamp_vec_per_axis(v, ref_point, bool_vec, frac=0.05):
    """
    Clamp displacement vector v per-axis so endpoint p0+v stays inside +/- frac*|p0_i| (plus eps).

    Args:
      v    : array-like shape (n,)
      ref_point   : array-like shape (n,)
      bool_vec : boolean vector for isolating categorical features out of changes
      frac : fraction (default 0.05)

    Returns:
      a clipped vector that is bounded within half of the interval on both sides for each dimension
    """
    # allowable bounds for the final point
    lower_bounds = (ref_point - (frac/2) * np.abs(ref_point)) * bool_vec
    upper_bounds = (ref_point + (frac/2) * np.abs(ref_point)) * bool_vec

    # clamp the endpoint
    endpoint = np.clip(ref_point + v, lower_bounds, upper_bounds)
    return np.clip(endpoint, lower_bounds, upper_bounds)

In [7]:
def run_dice_cfs(df, model, query_instance, method, continuous_features, categorical_features, target, chosen_row, contours, plot=False, total_CFs=1, delta=100):
    start = datetime.datetime.now()

    x_train = df.iloc[:, :-1]
    backend='sklearn'

    d = dice_ml.Data(dataframe=df, continuous_features=continuous_features, categorical_features=categorical_features, outcome_name=target)
    m = dice_ml.Model(model=model, backend=backend)

    exp_dice = dice_ml.Dice(d, m, method=method)
    
    query_instance = x_train.iloc[[chosen_row]].to_numpy()

    dice_cfs = exp_dice.generate_counterfactuals(pd.DataFrame(data=query_instance, columns=x_train.columns), 
                                                        total_CFs=total_CFs, desired_class="opposite")

    cfs_list = json.loads(dice_cfs.to_json())['cfs_list']
    dist_cfs = []

    # np_bounded_contour = np.array(bounded_contour_pts)  # Convert to NumPy array
    # x_values, y_values = np_bounded_contour[:, 0], np_bounded_contour[:, 1]  # Extract x/y for plotting
    # if plot:
    #     plt.scatter(x_values, y_values, marker='o')  # Plot bounded points
    contours = contours.reset_index(drop=True)

    bool_vec = []
    for col in df.iloc[:, :-1].columns:
        if np.issubdtype(df[col].dtype, np.number):
            bool_vec.append(1)   # numeric -> allow changes
        else:
            bool_vec.append(0)   # categorical -> mask out

    if delta == 100: 
        for point in cfs_list[0]: 
            point_vec = [float(point[i]) for i in range(len(point[:-1]))] 
            dist_cfs.append(euclidean_distance(np.array(point_vec), query_instance))
    else:
        for point in cfs_list[0]: 
            point_vec = [float(point[i]) for i in range(len(point[:-1]))]
            point_vec = np.reshape(np.array(point_vec), (1, -1))
            endpoint_vec = clamp_vec_per_axis(point_vec, ref_point=query_instance, bool_vec=bool_vec, frac=delta/100)
            closest_pt = closest_point(endpoint_vec, contour=contours.to_numpy())
            dist_cfs.append(euclidean_distance(closest_pt, endpoint_vec))

    # if plot:
    #     for point in cfs_list[0][:5]: 
    #         x,y = point[0], point[1] 
    #         print("EUCLIDEAN DISTANCE:", euclidean_distance(delta*np.array((x,y)), query_instance))
    #         plt.scatter(x,y, c = 'yellow')  # Plot optimal point (changed to green for distinction)
    #         plt.text(x+0.002, y+0.002, 'H')  # Label 'H' (e.g., Healthy; adjusted from duplicate 'NH')
    #         plt.plot([x,query_instance[0][0]], [y, query_instance[0][1]], linestyle='--')  # Dashed line between points
    end = datetime.datetime.now()
    diff = end - start
    print(f"Elapsed time: {diff}")

    return dist_cfs, diff.total_seconds()

In [19]:
def exps(dataset, model, method, target, x_train, y_train, continuous_features, categorical_features, inv_map, num_samples, delta=100, constraints=[], threshold=25000): 
    dice_dists, optimal_dists = [], [] 
    sub_dataset = dataset[dataset[target] == 1]
    random_integers = random.sample(range(0, sub_dataset.shape[0]-1), num_samples)
    
    for i in random_integers:
        real_idx = sub_dataset.index[i]
        chosen_row=real_idx
        query_instance=x_train.iloc[chosen_row:chosen_row+1,:]
        label = y_train.iloc[chosen_row:chosen_row+1]
        df, model, query_instance, opt_point, dist, exp_delta, boundary_points = optimal_point(dataset, model, desired_class=0, original_class=1, threshold=threshold, chosen_row=chosen_row, point_epsilon=1e-3, epsilon=0.01, constraints=constraints, delta=delta, step=0.1)
        optimal_dists.append(dist)
        dist_cfs, _ = run_dice_cfs(df=df, model=model, query_instance=query_instance,method=method, continuous_features=continuous_features, categorical_features=categorical_features, target=target, contours=boundary_points, chosen_row=chosen_row, delta=exp_delta, total_CFs=10)
        dice_dists.extend(dist_cfs)

# Step 3: Toy Dataset

We run a few experiments using the toy dataset, and we compare the results visually using both the optimal point method and the dice model-agnostic methods. 

In [8]:
inv_map = { 
    1: -1, 
    -1: 1
}
x_train = df.iloc[:,:-1]
y_train  = df.iloc[:,-1]
continuous_features=['x1', 'x2']
categorical_features=[]
target='y'

In [ ]:
optimal_dists, dice_dists = exps(df, model=svm_classifier, method='kdtree', 
                                 x_train=x_train, y_train=y_train, 
                                 continuous_features=continuous_features,
                                 categorical_features=categorical_features, 
                                 inv_map=inv_map, num_samples=500, 
                                 target=target)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 y
-1    10
 1    10
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(100, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.0704113  1.07024674]]
[[-2.60250521 -4.33750868]]


100%|██████████| 1/1 [00:00<00:00, 40.86it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:00.031828
Class counts:
 y
-1    10
 1    10
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(100, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.07245113 1.07155846]]
[[-0.4375329  -0.68757677]]


100%|██████████| 1/1 [00:00<00:00, 31.23it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


...

Elapsed time: 0:00:00.023087
Class counts:
 y
-1    10
 1    10
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(100, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.07677203 1.06926943]]
[[-0.15900278  1.46360111]]


100%|██████████| 1/1 [00:00<00:00, 46.96it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:00.025400
Class counts:
 y
-1    10
 1    10
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(100, 2)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.0855971  1.06806041]]
[[-0.06960249  0.55066252]]


100%|██████████| 1/1 [00:00<00:00, 56.06it/s]

Elapsed time: 0:00:00.022088


In [24]:
print(np.mean(optimal_dists), np.mean(dice_dists))

3.3127302428121985 8.472641279388391


In [ ]:
optimal_dists, dice_dists = exps(df, model=svm_classifier, method='random', 
                                 x_train=x_train, y_train=y_train, 
                                 continuous_features=continuous_features,
                                 categorical_features=categorical_features, 
                                 inv_map=inv_map, num_samples=500, 
                                 target=target)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

In [ ]:
optimal_dists, dice_dists = exps(df, model=svm_classifier, method='genetic', 
                                 x_train=x_train, y_train=y_train, 
                                 continuous_features=continuous_features,
                                 categorical_features=categorical_features, 
                                 inv_map=inv_map, num_samples=500, 
                                 target=target)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

# Step 3: Adult Income Dataset Experiments

We run a few experiments using the adult income dataset comparing DiCE model-agnostic methodologies and the Optimal Point method.

We follow the following steps: 

1. Import the dataset using the helpers function from DiCE 
2. Initialize the classifier which is a Random Forest Classifier in this case
3. Iterate for 50 or 100 randomly selected points using the Optimal point method 
4. After each iteration of generation with the optimal point method, we apply the run_dice_cfs method that enables us to generate counterfactuals using DiCE's specific model-agnostic approach

In [23]:
dataset = helpers.load_adult_income_dataset()

/home/danielsin/miniconda3/envs/gpu_env/lib/python3.12/site-packages/dice_ml/utils/helpers.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adult_data = adult_data.replace({'income': {'<=50K': 0, '>50K': 1}})


In [16]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [17]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = RandomForestClassifier()

In [24]:
x_train = dataset.iloc[:,:-1]
y_train = dataset.iloc[:,-1]
continuous_features=["age", "hours_per_week"]
categorical_features = ['marital_status', 'workclass', 'education', 'race', 'gender', 'occupation']
target='income'
inv_map = { 
    0: 1, 
    1: 0
}
constraints = [ 
    ("age", "equal"), 
    ("workclass", "equal"), 
    ("education", "equal"), 
    ("race", "equal"), 
    ("gender", "equal"), 
    ("occupation", "equal"), 
]

In [26]:
optimal_dists, dice_dists = exps(dataset, clf, 'kdtree', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500, delta=20, 
                                    constraints=constraints)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 income
0    19820
1    19820
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:378: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  select_pts = points.loc[points[constraint[0]] == undesired_coords[points.columns.get_loc(constraint[0])], :]


boundary points finished.
(50000, 8)
[[0.         0.         0.         1.00334448 0.         0.
  0.         0.99960016]]
[[ 0.   0.   0.  -0.3  0.   0.   0.   2.5]]


100%|██████████| 1/1 [00:00<00:00,  1.45it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


...

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:378: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  select_pts = points.loc[points[constraint[0]] == undesired_coords[points.columns.get_loc(constraint[0])], :]


boundary points finished.
(50000, 8)
[[0.         0.         0.         0.99750623 0.         0.
  0.         1.00033344]]
[[ 0.   0.   0.   0.4  0.   0.   0.  -3. ]]


100%|██████████| 1/1 [00:01<00:00,  1.09s/it]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:01.264996
Class counts:
 income
0    19820
1    19820
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...


KeyboardInterrupt: 

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

In [ ]:
optimal_dists, dice_dists = exps(dataset, clf, 'random', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500, delta=20, 
                                    constraints=constraints)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

In [ ]:
optimal_dists, dice_dists = exps(dataset, clf, 'genetic', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500, delta=20, 
                                    constraints=constraints)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

# Step 3: Heart Disease Dataset Experiments

We run a few experiments using the heart disease dataset comparing DiCE model-agnostic methodologies and the Optimal Point method.

We follow the following steps: 

1. Import the dataset using the helpers function from DiCE 
2. Initialize the classifier which is a Random Forest Classifier in this case
3. Iterate for 50 or 100 randomly selected points using the Optimal point method 
4. After each iteration of generation with the optimal point method, we apply the run_dice_cfs method that enables us to generate counterfactuals using DiCE's specific model-agnostic approach

In [20]:
heart_disease = pd.read_csv(
'../../heart.csv'
)

In [21]:
print(heart_disease.dtypes)

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object


In [22]:
constraints = [ 
    ("age", "equal"),
    ("sex", "equal"), 
    ("cp", "equal"), 
    ("fbs", "equal"), 
    ("restecg", "equal"), 
    ("exang", "equal"), 
    ("slope", "equal"), 
    ("thal", "equal")
]

In [23]:
x_train = heart_disease.iloc[:,:-1]
y_train = heart_disease.iloc[:,-1]
continuous_features=["age", "trestbps", "thalach", "oldpeak", "chol"]
categorical_features=['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
target='target'
inv_map = { 
    0: 1, 
    1: 0
}

In [ ]:
optimal_dists, dice_dists = exps(dataset=heart_disease, model=svm_classifier, method='kdtree', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500, delta=20, 
                                    constraints=constraints)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 target
0    526
1    526
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(25000, 13)


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:378: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  select_pts = points.loc[points[constraint[0]] == undesired_coords[points.columns.get_loc(constraint[0])], :]


[[0.         0.         0.         0.99954566 1.00027034 0.
  0.         1.00038476 0.         1.01010101 0.         0.99009901
  0.        ]]
[[ 0.   0.   0.   2.2 -3.7  0.   0.  -2.6  0.  -0.1  0.   0.1  0. ]]


100%|██████████| 1/1 [00:00<00:00,  3.88it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


...

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:378: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  select_pts = points.loc[points[constraint[0]] == undesired_coords[points.columns.get_loc(constraint[0])], :]


[[0.         0.         0.         0.         1.00027034 0.
  0.         1.00502513 0.         0.98765432 0.         1.01010101
  0.        ]]
[[ 0.    0.    0.    0.   -3.7   0.    0.   -0.2   0.    0.08  0.   -0.1
   0.  ]]


100%|██████████| 1/1 [00:00<00:00,  4.90it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:00.270461
Class counts:
 target
0    526
1    526
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(25000, 13)


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:378: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  select_pts = points.loc[points[constraint[0]] == undesired_coords[points.columns.get_loc(constraint[0])], :]


In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

In [ ]:
optimal_dists, dice_dists = exps(dataset=heart_disease, model=svm_classifier, method='random', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500, delta=20, 
                                    constraints=constraints)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

In [ ]:
optimal_dists, dice_dists = exps(dataset=heart_disease, model=svm_classifier, method='genetic', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500, delta=20, 
                                    constraints=constraints)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

# Comparing DiCE with large feature dataset made of numerical features 

We use the ```make_classification``` function of sci-kit learn library to generate a synthetic dataset of numerical values that we can then compare both methodologies.

In [148]:
X, y = make_classification(n_samples=2000, n_features=20, n_informative=20, n_redundant=0, random_state=42, n_classes=2)
y = y.reshape(-1,1)
columns = ["x"+str(i) for i in range(20)] 
columns.append('y')
dataset = pd.DataFrame(data=np.hstack((X,y)), columns=columns) 
model = LogisticRegression()
continuous_features = columns[:-1]
categorical_features=[]
target = 'y'
x_train = dataset.iloc[:,:-1]
y_train = dataset.iloc[:,-1]

In [150]:
optimal_dists, dice_dists = exps(dataset=dataset, model=model, 
                                 method='kdtree', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500)

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


(1005, 21)
            x0        x1        x2        x3        x4        x5        x6  \
0    -1.453777  1.655735  1.329367  1.819664  1.314480  1.268243 -2.386854   
1     2.655292  1.411449  3.029983  0.846878 -4.143281  0.025324  0.697482   
2     4.106648 -0.775428  2.117885  0.488454  0.402942  0.257583 -2.627793   
3     2.831730  0.631128  5.007239 -1.861998  2.549420 -0.462646 -1.869716   
4     2.226998 -0.769184  4.072671 -4.275346  3.952438 -1.238654 -0.862764   
...        ...       ...       ...       ...       ...       ...       ...   
1995 -4.094447  3.347500  2.573580 -1.615300 -4.883879 -1.577929 -2.805046   
1996 -3.392901 -0.729040  3.245343 -1.426268 -1.186759 -1.147924 -1.992854   
1997  5.990654 -1.243620 -1.484536  0.544669  0.125498  2.416384 -0.206431   
1998 -0.956687  0.555954  2.253433  2.055390 -0.488982  1.153996  1.737616   
1999 -8.657856 -2.221471 -1.975641  5.674702  1.526432 -4.156842 -0.879709   

            x7        x8        x9  ...       x11   

  0%|          | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

In [ ]:
optimal_dists, dice_dists = exps(dataset=dataset, model=model, 
                                 method='random', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

In [ ]:
optimal_dists, dice_dists = exps(dataset=dataset, model=model, 
                                 method='genetic', target=target,
                                  x_train=x_train, y_train=y_train, 
                                  continuous_features=continuous_features,
                                    categorical_features=categorical_features, 
                                    inv_map=inv_map, num_samples=500)

In [ ]:
print(np.mean(optimal_dists), np.mean(dice_dists))

# Step 4: Runtime Tests 

The function ```runtime_tests()``` are used for comparing DiCE's model-agnostic approaches and Optimal Point for time complexity. We use a logistic regression classifier for examining runtime.

In [12]:
def runtime_tests(number_of_features, method, total_random=100):
    X, y = make_classification(n_samples=5000, n_features=number_of_features, n_informative=number_of_features,
                            n_redundant=0, n_classes=2, random_state=42)
    y = np.reshape(y, (-1, 1))
    columns = ["x"+str(i) for i in range(1, X.shape[1]+1)] 
    columns.append('y')
    dataset = pd.DataFrame(data=np.hstack((X,y)), columns=columns)
    continuous_features=["x"+str(i) for i in range(1, X.shape[1]+1)]
    target='y'
    inv_map = { 
        0: 1, 
        1: 0
    }
    dice_dists, optimal_dists = [], []
    dice_runtime = [] 
    sub_dataset = dataset[dataset[target] == 0]
    random_integers = random.sample(range(1, sub_dataset.shape[0]), total_random)
    clf = LogisticRegression()

    for i in random_integers:
        real_idx = sub_dataset.index[i]
        chosen_row=real_idx
        query_instance=X[chosen_row:chosen_row+1,:]
        label = y[chosen_row:chosen_row+1]
        df, model, query_instance, opt_point, dist,_, contours = optimal_point(dataset, clf, desired_class=inv_map[label.item()], original_class=label.item(), threshold=5000, chosen_row=chosen_row, point_epsilon=1e-3, epsilon=0.01, constraints=[])
        optimal_dists.append(dist)
        dist_cfs, total_seconds = run_dice_cfs(df=df, contours=contours, model=model,query_instance=query_instance,method=method, continuous_features=continuous_features, categorical_features=[], target=target, chosen_row=chosen_row)
        dice_dists.extend(dist_cfs)
        dice_runtime.append(total_seconds)

    print(np.mean(dice_runtime))

In [13]:
runtime_tests(number_of_features=10, method='kdtree')

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[0.97875647 1.01764626 0.97893963 0.96303197 1.00476183 1.00160176
  1.00024438 0.98786358 1.00254436 1.02078757]]
[[ 0.03132669 -0.13309066  0.03151732  0.02050399  0.1918156   0.11926333
   0.10253012  0.04462616  0.13446786 -0.0946263 ]]


100%|██████████| 1/1 [00:00<00:00, 19.91it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:00.061635
Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01488069 1.01203919 1.01449488 0.95870549 1.00848681 1.02417262
  1.00803271 1.00888745 1.00943566 1.0062081 ]]
[[-0.20793781 -0.49629488 -0.22569997  0.01869021  0.66646492 -0.07226416
   0.51239694  0.90682703  1.78869666  0.26535692]]


100%|██████████| 1/1 [00:04<00:00,  4.82s/it]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


...

Elapsed time: 0:00:04.680373
Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01054151 1.01065511 1.0091161  0.99545437 1.01453632 1.00937822
  1.00944021 1.00967765 1.00874974 1.01091053]]
[[-1.86613817 -1.5427313   1.14166005  0.06843666 -0.22364749  1.62336529
   1.80325469  3.13222821  0.8068309  -1.11024556]]


100%|██████████| 1/1 [00:04<00:00,  4.73s/it]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:04.734286
Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01096937 1.00912706 1.01062486 1.01094106 1.00885938 1.01181636
  1.0113154  1.01470126 1.0086049  1.00890751]]
[[-1.04291147  1.15600697 -1.61735015 -1.07425257  0.8844798  -0.55705651
  -0.76882622 -0.2158362   0.72296098  0.92349159]]


  0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
runtime_tests(number_of_features=50, method='kdtree')

In [14]:
runtime_tests(number_of_features=10, method='random')

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[0.48006337 0.98612372 1.01065299 1.00424052 1.00928379 1.0140577
  1.00855108 1.00774581 1.01198776 1.00918467]]
[[ 9.05888259e-04  4.13013971e-02 -1.54772298e+00  1.74363086e-01
   1.40919223e+00 -2.49909785e-01  6.96073114e-01  4.47055407e-01
  -5.09110272e-01  1.23776387e+00]]


100%|██████████| 1/1 [00:00<00:00, 13.13it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:00.084478
Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01110379 1.01067849 1.01060623 1.00760008 1.00840962 1.01119239
  1.02753227 1.00943864 1.01120208 1.00931206]]
[[-0.91602905 -1.48959798 -1.66704373  0.41984694  0.63406745 -0.84804015
  -0.05860804  1.79820421 -0.84121216  1.46715474]]


100%|██████████| 1/1 [00:00<00:00, 13.46it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


...

Elapsed time: 0:00:00.078360
Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01179775 1.01036065 1.01070383 1.01054418 1.00963383 1.00948603
  1.01056219 1.0095895  1.01515863 1.00951094]]
[[-0.56281467 -2.80149128 -1.43601027 -1.85699342  2.75725784  1.96408664
  -1.7975616   2.45940509 -0.19678852  2.06419503]]


100%|██████████| 1/1 [00:00<00:00, 15.30it/s]

Elapsed time: 0:00:00.072399
0.20418605


In [ ]:
runtime_tests(number_of_features=50, method='random')

In [15]:
runtime_tests(number_of_features=10, method='genetic')

/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01062175 1.01072536 1.01026308 1.00942177 1.00935965 1.00977479
  1.00920245 0.98552083 1.00939383 1.01079159]]
[[-1.62545712 -1.39340853 -3.84009716  1.74572365  1.57626569  4.48368444
   1.26537876  0.04025957  1.665194   -1.27690528]]


100%|██████████| 1/1 [00:00<00:00, 26.48it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Elapsed time: 0:00:00.044273
Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.01168475 1.01084654 1.01133742 1.01158642 0.99777639 1.01193077
  1.01778186 1.00947919 1.00651397 1.0085238 ]]
[[-0.60049593 -1.19409856 -0.75618455 -0.63765461  0.08162695 -0.52410839
  -0.13078895  1.93828909  0.28872815  0.68318915]]


100%|██████████| 1/1 [00:00<00:00, 29.73it/s]
/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


...

Elapsed time: 0:00:00.041952


/mnt/c/Users/Compuworld/drexel_research_2024_2025/Docs/files/common_functions.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_dataset = pd.concat([balanced_dataset, upsampled_class], ignore_index=True)


Class counts:
 y
1    2511
0    2511
Name: count, dtype: int64
Fitting model...
Model training complete.
boundary points started generation...
boundary points finished.
(5000, 10)
Finding the closest point from the contour line to the point...
Found the closest point from the contour line to the point.
[[1.42178366 1.02466628 1.02873396 0.98446763 1.03770311 1.02937749
  1.10050369 0.91054725 0.99939485 0.98065764]]
[[-0.00345274 -0.06986546 -0.0549128   0.03855762 -0.037458   -0.05312234
  -0.01215977  0.00915558  0.09423675  0.03342123]]


100%|██████████| 1/1 [00:00<00:00, 34.09it/s]

Elapsed time: 0:00:00.034917
0.04050603


In [ ]:
runtime_tests(number_of_features=50, method='genetic')